# Python HITMIX Example: Text Analysis

In this notebook, we demonstrate the use of HITMIX for computing _hitting time moments_ for a set of vertices in a text analysis application.

```
Copyright 2021 National Technology & Engineering Solutions of Sandia,
LLC (NTESS). Under the terms of Contract DE-NA0003525 with NTESS, the
U.S. Government retains certain rights in this software.
```

## Authors
* Danny Dunlavy (dmdunla@sandia.gov)
* Peter Chew (pachew@galiteoconsulting.com)

## Load package dependencies

In [ ]:
import os
import time

import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix, coo_matrix
from scipy.sparse.linalg import svds
from sklearn.feature_extraction.text import CountVectorizer

# Plotting
import matplotlib.pyplot as plt
%matplotlib inline

# NLTK -- used in Gensim for the Lemmatizer and Stemmers. 
import nltk

# Regular expressions
import re

## Setup data and work directories

In [ ]:
# For local import of the data
data_dir_name    = "./data/"
figures_dir_name = "./figures/"
models_dir_name  = "./models/"

dirs = [data_dir_name, figures_dir_name, models_dir_name]
for dr in dirs: 
    if not os.path.exists(dr):
        os.makedirs(dr)

## Import data into Pandas DataFrame
See [README.md](/edit/README.md) file for information about downloading and using these datasets.

NOTE: Uncomment lines of datasets you want to use; comment lines of datasets you do not want to use.

In [ ]:
# Multiline ASCII text
corpus_filename = "multiline-ascii.txt" 
df = pd.read_csv(data_dir_name + corpus_filename, header=None, names=['text'])
# replace line feeds with spaces in multiline text
df['text'] = df['text'].str.replace('\n',' ')
df['text'] = df['text'].str.replace('\r',' ')

# # UTF-8 data in multiple languages
# corpus_filename = "language-learning-and-teaching.txt" 
# df = pd.read_csv(data_dir_name + corpus_filename, header=None, sep='\n', names=['text'])

# # Twitter BBC health news headlines
# corpus_filename = "bbchealth.txt" 
# df = pd.read_csv(data_dir_name + corpus_filename, header=None, sep='|', names=['id','date','text'])

In [ ]:
df

## Create the Document-Term Matrix using the Sklearn CountVectorizer

In [ ]:
# preprocessor to remove various unwanted terms
def preprocess_text(text):
    text = text.lower()
    # remove URLs
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','',text)
    # remove numbers
    #text = re.sub(r'\d+', '', text)
    return text

start = time.time()

# create document-term matrix
corpus = df["text"]
# Alternative: strip accents, min document frequency = 1%, max document frequency = 75%
#vectorizer = CountVectorizer(preprocessor=preprocess_text, strip_accents='ascii', min_df=int(.01*len(df)), max_df=int(.75*len(df)))
vectorizer = CountVectorizer(preprocessor=preprocess_text)
A = vectorizer.fit_transform(corpus)

end = time.time()
fit_transform_time = end-start
print(f"It took {end-start} seconds to compute the document-term matrix")
print(f"A: {A.shape}")
dictionary = np.array(vectorizer.get_feature_names())
# print vocabulary as a dictionary (term : index)
#print(vectorizer.vocabulary_)

## Pointwise Mutual Information (PMI) between terms and documents

In [ ]:
def matrix_pmi(A):
    # pmi(i,j) = log2( p(i,j)/p(i)/p(j) ) for term i in document j

    # probability of term i over corpus: p(i)
    term_sums = A.sum(axis=0)
    total_terms = term_sums.sum()
    p_i = term_sums / total_terms

    # probability of term i over corpus: p(i)
    doc_sums = A.sum(axis=1)
    p_j = doc_sums / doc_sums.sum() 

    # weighted matrix
    A_pmi = A.copy()
    A_pmi.data = A_pmi.data.astype(float)
    pmi= (A_pmi/total_terms)/p_i/p_j
    A_pmi[A.nonzero()] = np.log2(pmi[A.nonzero()])
    
    return A_pmi

In [ ]:
# validate matrix_pmi

# Validation uses data from Table 2 in the following journal article:
# CHEW, P., BADER, B., HELMREICH, S., ABDELALI, A., & VERZI, S. (2011). 
# An information-theoretic, vector-space-model approach to cross-language 
# information retrieval. Natural Language Engineering, 17(1), 37-70. 
# doi:10.1017/S1351324910000185
Atest_term_counts = csr_matrix([[2,1,1,1,0,0,0,0],[3,1,0,1,1,1,1,0],[2,0,0,1,0,1,1,1]])

# Compute PMI
Atest_pmi = matrix_pmi(Atest_term_counts)

# Round the data to 3 digits of accuracy
Atest_pmi.data = np.round(Atest_pmi.data,3)

# Convert to dense term-document matrix (i.e., transpose)
Atest_pmi_final = Atest_pmi.T.todense()

# Table 3 data from the same journal article:
Atrue_pmi = np.array([[ 0.119,  0.026, -0.144],
                      [ 0.926,  0.248,  0.   ],
                      [ 1.926,  0.,     0.   ],
                      [ 0.341, -0.337,  0.078],
                      [ 0.,     1.248,  0.   ],
                      [ 0.,     0.248,  0.663],
                      [ 0.,     0.248,  0.663],
                      [ 0.,     0.,     1.663]])

# check if they are the same
if np.linalg.norm(Atest_pmi_final-Atrue_pmi) == 0:
    print("The computed and published PMI values match!")
else:
    print("The computed and published PMI values do not match!")

In [ ]:
# compute PMI and show the min and max values
A_pmi = matrix_pmi(A)
print(f"Min PMI: {np.min(A_pmi)}, Max PMI: {np.max(A_pmi)}")

## Plot histogram of PMI weights

In [ ]:
plt.figure()
plt.hist(A_pmi.data, log=True, bins=100)
plt.xlabel('PMI')
plt.ylabel('Count')
plt.show()

## Compute a truncated singular value decomposition (SVD) of the weighted document-term matrix 

In [ ]:
# user-defined topics (i.e., rank of SVD)
num_topics = min([min([A_pmi.shape[0], A_pmi.shape[1]])-1, 100])
print(f"Number of topics to compute: {num_topics}")

# compute SVD
U,S,Vt = svds(A_pmi, k=num_topics)

# reorder SVD based on decreasing singular values
U[:,:num_topics] = U[:, num_topics-1::-1]
S = S[::-1]
Vt[:num_topics, :] = Vt[num_topics-1::-1, :]

# plot distribution of singular values
plt.figure()
plt.plot(range(len(S)), S, '.')
plt.xlabel('Index')
plt.ylabel('Singular Value')
plt.show()

## Plot explained variance for each topic (over all data and topics)

_Note:_ For large document-term matrices, the following computations may take a very long time.

In [ ]:
#Ufull,Sfull,Vtfull = svds(A_pmi, k=min(A_pmi.shape)-1)

In [ ]:
# sum_sv = Sfull.sum()
# plt.plot(range(len(Sfull)), 100*Sfull[::-1]/sum_sv, 'ko')
# plt.xlabel('Index of topic')
# plt.ylabel('Percentage of explained variance')
# plt.show()

In [ ]:
# sum_sv = Sfull.sum()
# plt.plot(range(len(Sfull)), 100*Sfull[::-1].cumsum()/sum_sv, 'ko')
# plt.xlabel('Index of topic')
# plt.ylabel('Cummulative percentage of explained variance')
# plt.show()

## Topic-Term-Document Helper Functions

In [ ]:
def get_topic_terms(Vt, t, n):
    # get term vector for topic t
    term_dist = Vt[t, :]
    # sort the term vector weights
    sort_indices = np.flip(term_dist.argsort())
    terms = dictionary[sort_indices][:n]
    weights = term_dist[sort_indices][:n]
    # return top n terms as a list
    return terms, weights

def get_topic_documents(U, t, n):
    # get document vector for topic t
    doc_dist = U[:, t]
    # sort the document vector weights
    sort_indices = np.flip(doc_dist.argsort())
    docs = corpus[sort_indices][:n]
    weights = doc_dist[sort_indices][:n]
    # return top n docs as a list
    return docs, weights

def get_document_topics(U, d):
    # get topic vector for document d
    doc_dist = U[d, :]
    # return topic vector
    return doc_dist

## Plot topic-term distributions

In [ ]:
N = A.shape[1]
topics_to_plot = [0,1]
plt.figure()
for i in topics_to_plot:
    terms, weights = get_topic_terms(Vt, i, N)
    plt.plot(weights,'.')
plt.legend(["Topic "+str(i) for i in topics_to_plot],bbox_to_anchor=(1,1),loc='upper left')
plt.ylabel("term weight")
plt.xlabel("term index")
plt.show()

## Plot document-topic distributions

In [ ]:
N = A.shape[0]
docs_to_plot = [0,1,2]
plt.figure()
for i in docs_to_plot:
    doc_dist = get_document_topics(U, i)
    plt.plot(doc_dist,'.')
plt.legend(["Document "+str(i) for i in docs_to_plot],bbox_to_anchor=(1,1),loc='upper left')
plt.ylabel("topic weight")
plt.xlabel("topic index")
plt.show()

## Plot topic-document distributions

In [ ]:
import matplotlib
num_docs = U.shape[0]
doc_list = np.array(range(num_docs))
colors = ["black", "red"]
# all docs are black
color_indices = np.zeros((num_docs))
# the last 2 docs are red
color_indices[-2:] = 1
colormap = matplotlib.colors.ListedColormap(colors)

subplot_columns = 4
plt.figure(figsize=(20,2*num_topics//subplot_columns + 1))
for i in range(num_topics):
    plt.subplot(num_topics // subplot_columns + 1, subplot_columns, i + 1)
    plt.scatter(doc_list,U[:, i],c=color_indices,cmap=colormap,marker='.')
    plt.gca().get_xaxis().set_visible(False)
    plt.gca().get_yaxis().set_visible(False)
    plt.title(f"topic {i}")
plt.show()

## Compute graph adjacency matrix

In [ ]:
# flag to control normalizing (2-norm unit length) rows of scaled left singular vectors
scale_US = True

# scale document vectors with singular values
US = U@np.diag(np.sqrt(S))

# normalize rows of US is requested
if (scale_US):
    US = US / np.linalg.norm(US,axis=1)[:, np.newaxis]

print(f"Row norms of US: mean={np.mean(np.linalg.norm(US,axis=1)):.4f}; std={np.std(np.linalg.norm(US,axis=1)):.4f}")
    
# construct similarity graph weighted adjacency matrix, zero out the negatives
Adj = US @ US.T
Adj[Adj <= 0] = 0
Adj_sparse = csr_matrix(Adj)
print(f"Adjacency shape: {Adj_sparse.shape}")
print(f"Adjacency nonzeros: {Adj_sparse.nnz}")
print(f"Adjacency density: {Adj_sparse.nnz/np.prod(Adj_sparse.shape)}")



## Plot distribution of adjacency matrix edge weights

In [ ]:
plt.figure()
plt.hist(Adj_sparse.data,log=True, bins=200)
plt.xlabel('Edge weight')
plt.ylabel('Count')
plt.show()

## Threshhold adjacency matrix and just save topology

In [ ]:
# Useful thresholds will depend on scaling of singular vectors used in creating the 
# adjacency graph. The plot above is useful for finding a useful threshold via visual 
# inspection of the edge weights.
threshold = 0.5
Adj_thresh = Adj_sparse.copy()
Adj_thresh.data = np.where(Adj_thresh.data < threshold, 0, Adj_thresh.data)
Adj_thresh.eliminate_zeros()
#Adj_thresh.data[:] = 1 
print(f"Adjacency shape: {Adj_thresh.shape}")
print(f"Adjacency nonzeros: {Adj_thresh.nnz}")
print(f"Adjacency density: {Adj_thresh.nnz/np.prod(Adj_thresh.shape)}")

## View top terms and documents per topic 

Print out a few documents that are the most representative of each topic

In [ ]:
num_terms_per_topic_to_display = 10
num_docs_per_topic_to_display = 3
num_chars_per_doc_to_display = 80
include_topic_weights = True

for t in range(num_topics):
    # get terms and topic weights
    terms, weights = get_topic_terms(Vt,t,num_terms_per_topic_to_display)

    if include_topic_weights:
        # include weights
        terms_list = [f'{terms[i]} ({weights[i]:1.3f})' for i in range(len(terms))]
        terms_str = f"{t} ({S[t]:0.3f}): " + ' '.join(terms_list)
    else:
        # don't include weights
        terms_list = [f'{terms[i]} ' for i in range(len(terms))]
        terms_str = f"{t}: " + ' '.join(terms_list)
    
    print(f"{terms_str}\n")
    docs, doc_weights = get_topic_documents(U, t, num_docs_per_topic_to_display)
    for doc_index, doc in docs.items():
        if include_topic_weights:
            print(f"\tDoc {doc_index} ({U[doc_index,t]:1.3f}) {doc[:num_chars_per_doc_to_display]}")
        else:
            print(f"\tDoc {doc_index} {doc[:num_chars_per_doc_to_display]}")
    print('\n')


## Choose subset of documents as the hitting set

In [ ]:
# manually selected after reading documents in different topics above
documents_of_interest = sorted([1, 0])
print(documents_of_interest)

## Compute mean hitting times to `documents_of_interest` (hitting set)

In [ ]:
from hitmix import hitting_time_moments
ETm, cg_info = hitting_time_moments(Adj_thresh, documents_of_interest, maxiter=200)

## Provide rank order of documents related to hitting set

Hitting time moments values of 0 indicate vertices in the hitting set.

Hitting time moments values of $\infty$ indicate vertices not related to (i.e. not reachable in the graph from) the hitting set.


In [ ]:
rank_order = ETm[:,0].argsort()
for i in range(ETm.shape[0]):
    print(i, rank_order[i], ETm[rank_order][i,0], corpus[rank_order[i]][0:100])

In [ ]:
# add mean hitting times to the original DataFrame and print it out
df['ETm[0]'] = ETm[:,0]
df